In [1]:
%matplotlib inline
import pylab as plt
from IPython.display import clear_output
import numpy as np
import asyncio

In [ ]:
import pandas as pd  
import numpy as np  

# 从CSV文件读取数据  
# 请确保路径和文件名与实际一致  
df = pd.read_csv('spotdis.csv', encoding='utf-8')  # 请根据需要调整编码  

# 创建一个包含城市名称的所有唯一值的列表  
all_cities = pd.concat([df['出发景点'], df['目的景点']]).unique()  

# 创建城市到索引的映射  
city_to_index = {city: idx for idx, city in enumerate(all_cities)}  

# 初始化距离矩阵  
num_cities = len(all_cities)  
distance_matrix = np.full((num_cities, num_cities), np.inf)  # 使用无穷大填充  

# 填充距离矩阵  
for idx, row in df.iterrows():  
    start = city_to_index[row['出发景点']]  
    end = city_to_index[row['目的景点']]  
    distance_matrix[start, end] = row['驾车距离（公里）']  
    distance_matrix[end, start] = row['驾车距离（公里）']  # 设置反向距离  


# 替换对角线为0（自己到自己距离为0）  
np.fill_diagonal(distance_matrix, 0)  

# 将距离矩阵转换为 DataFrame 方便查看  
distance_matrix_df = pd.DataFrame(distance_matrix, index=all_cities, columns=all_cities)  

print(city_to_index)

{'五台山': 0, '云冈石窟': 1, '晋祠博物馆': 2, '应县木塔': 3, '山西黄河壶口瀑布旅游区': 4, '平遥古城': 5, '小西天': 6, '太原植物园': 7, '广胜寺': 8, '山西王家大院': 9, '皇城相府': 10, '运城关公故里文化旅游景区': 11, '雁门关': 12, '太原动物园': 13, '仙堂山': 14, '华门': 15, '大阳古镇': 16, '柳氏民居': 17, '河边民俗博物馆': 18, '金沙滩生态旅游区': 19, '海会书院': 20, '舜帝陵': 21, '人祖山景区': 22, '赵杲观景区': 23, '望仙大峡谷': 24, '青草坡乡村庄园': 25, '赤壁悬流': 26, '湘峪村': 27, '石膏山': 28, '中太行洗耳河景区': 29, '平型关大捷景区': 30, '八路军太行纪念馆': 31, '珏山': 32, '欢乐太行谷': 33, '天宁寺': 34}


In [85]:
import pandas as pd


class TSP(object):
    '''
    用 Q-Learning 求解 TSP 问题
    作者 Surfer Zen @ https://www.zhihu.com/people/surfer-zen
    '''
    def __init__(self, 
                 path,
                 num_cities=35, 
                 map_size=(0.0, 1.0), 
                 alpha=2, 
                 beta=1,
                 learning_rate=0.001,
                 eps=0.1,
                ):
        '''
        Args:
            num_cities (int): 城市数目
            map_size (int, int): 地图尺寸（宽，高）
            alpha (float): 一个超参，值越大，越优先探索最近的点
            beta (float): 一个超参，值越大，越优先探索可能导向总距离最优的点
            learning_rate (float)： 学习率
            eps (float): 探索率，值越大，探索性越强，但越难收敛 
        '''
        self.num_cities =num_cities
        self.map_size = map_size
        self.alpha = alpha
        self.beta = beta
        self.eps = eps
        self.learning_rate = learning_rate
        self.file_path=path
        # self.cities = self.generate_cities()
        # self.cities=self.load_and_normalize_coordinates(self.file_path)
        self.distances = self.get_dist_matrix(path)
        self.mean_distance = self.distances.mean()
        self.qualities = np.zeros([num_cities, num_cities])
        self.normalizers = np.zeros(num_cities)
        self.best_path = None
        self.best_path_length = np.inf
        self.city_to_index={}


    def generate_cities(self):
        '''
        随机生成城市（坐标）
        Returns:
            cities: [[x1, x2, x3...], [y1, y2, y3...]] 城市坐标
        '''
        max_width, max_height = self.map_size
        cities = np.random.random([2, self.num_cities]) \
            * np.array([max_width, max_height]).reshape(2, -1)
        return cities

    
    def get_dist_matrix(self,file_path):
    #     '''
    #     根据城市坐标，计算距离矩阵
    #     '''
    #     dist_matrix = np.zeros([self.num_cities, self.num_cities])
    #     for i in range(self.num_cities):
    #         for j in range(self.num_cities):
    #             if i == j:
    #                 continue
    #             xi, xj = self.cities[0, i], self.cities[0, j]
    #             yi, yj = self.cities[1, i], self.cities[1, j]
    #             dist_matrix[i, j] = np.sqrt((xi-xj)**2 + (yi-yj)**2)
    #     return dist_matrix
        '''  
        得到距离矩阵  
        
        :param file_path: CSV 文件路径  
        :return: 距离矩阵的 DataFrame  
        '''  
        # 从CSV文件读取数据  
        df = pd.read_csv(file_path, encoding='utf-8')  # 请根据需要调整编码  
    
        # 创建一个包含城市名称的所有唯一值的列表  
        all_cities = pd.concat([df['出发景点'], df['目的景点']]).unique()  
    
        # 创建城市到索引的映射  
        city_to_index = {city: idx for idx, city in enumerate(all_cities)}  
        self.city_to_index = city_to_index
    
        # 初始化距离矩阵  
        num_cities = len(all_cities)  
        distance_matrix = np.full((num_cities, num_cities), np.inf)  # 使用无穷大填充  
    
        # 填充距离矩阵  
        for idx, row in df.iterrows():  
            start = city_to_index[row['出发景点']]  
            end = city_to_index[row['目的景点']]  
            distance_matrix[start, end] = row['驾车距离（公里）']  
            distance_matrix[end, start] = row['驾车距离（公里）']  # 设置反向距离  
    
        # 替换对角线为0（自己到自己距离为0）  
        np.fill_diagonal(distance_matrix, 0) 
    
        # 将距离矩阵转换为 DataFrame 方便查看  
        distance_matrix_df = pd.DataFrame(distance_matrix, index=all_cities, columns=all_cities)  
    
        print(distance_matrix_df) 


        return distance_matrix
    
    def get_best_path(self):  
        '''  
        获取最佳路径及其长度  
        Returns:  
            best_path: 最佳路径  
            best_path_length: 最佳路径长度  
        '''  
        return self.best_path, self.best_path_length  
        

    def rollout(self, start_city_id=None):  
        cities_visited = []  
        action_probs = []  
        if start_city_id is None:  
            start_city_id = np.random.randint(self.num_cities)  
        current_city_id = start_city_id  
        cities_visited.append(current_city_id)  
        
        while len(cities_visited) < self.num_cities:  
            current_city_id, action_prob = self.choose_next_city(cities_visited)  
            cities_visited.append(current_city_id)  
            action_probs.append(action_prob)  
        
        cities_visited.append(cities_visited[0])  # 回到起点  
        action_probs.append(1.0)  
    
        path_length = self.calc_path_length(cities_visited)  
        print(f"Visited Cities: {cities_visited}, Path Length: {path_length}")  # 调试输出  
    
        if path_length < self.best_path_length:  
            self.best_path = cities_visited  
            self.best_path_length = path_length  
            print(f"New Best Path: {self.best_path}, Length: {self.best_path_length}")  # 调试输出  
    
        rewards = self.calc_path_rewards(cities_visited, path_length)  
        print(f"Rewards: {rewards}")  # 调试输出  
        return cities_visited, action_probs, rewards

    def choose_next_city(self, cities_visited):
        '''
        根据策略选择下一个城市
        '''
        current_city_id = cities_visited[-1]
        
        # 对 quality 取指数，计算 softmax 概率用
        probabilities = np.exp(self.qualities[current_city_id])

        # 将已经走过的城市概率设置为零
        for city_visited in cities_visited:
            probabilities[city_visited] = 0

        # 计算 softmax 概率
        probabilities = probabilities/probabilities.sum()
        #     # 确保概率总和为1  
        # total_prob = probabilities.sum()  
        # if total_prob == 0:  
        #     probabilities = np.ones_like(probabilities) / len(probabilities)  
        # else:  
        #     probabilities /= total_prob  
        
        if np.random.random() < self.eps:
            # 以 eps 概率按softmax概率密度进行随机采样
            next_city_id = np.random.choice(range(len(probabilities)), p=probabilities)
        else:
            # 以 (1 - eps) 概率选择当前最优策略
            next_city_id = probabilities.argmax()

        # 计算当前决策/action 的概率
        if probabilities.argmax() == next_city_id:
            action_prob = probabilities[next_city_id]*self.eps + (1-self.eps)
        else:
            action_prob = probabilities[next_city_id]*self.eps
            
        return next_city_id, action_prob

    def calc_path_rewards(self, path, path_length):
        '''
        计算给定路径的奖励/rewards
        Args:
            path (list[int]): 路径，每个元素代表城市的 id
            path_length (float): 路径长路
        Returns:
            rewards: 每一步的奖励，总距离以及当前这一步的距离越大，奖励越小
        '''
        rewards = []
        for fr, to in zip(path[:-1], path[1:]):
            dist = self.distances[fr, to]
            reward = (self.mean_distance/path_length)**self.beta
            reward = reward*(self.mean_distance/dist)**self.alpha
            rewards.append(np.log(reward))
        return rewards

    def calc_path_length(self, path):
        '''
        计算路径长度
        '''
        path_length = 0
        for fr, to in zip(path[:-1], path[1:]):
            path_length += self.distances[fr, to]
        return path_length
    
    def calc_updates_for_one_rollout(self, path, action_probs, rewards):
        '''
        对于给定的一次 rollout 的结果，计算其对应的 qualities 和 normalizers 
        '''
        qualities = []
        normalizers = []
        for fr, to, reward, action_prob in zip(path[:-1], path[1:], rewards, action_probs):
            log_action_probability = np.log(action_prob)
            qualities.append(- reward*log_action_probability)
            normalizers.append(- (reward + 1)*log_action_probability)
        return qualities, normalizers

    def update(self, path, new_qualities, new_normalizers):
        '''
        用渐近平均的思想，对 qualities 和 normalizers 进行更新
        '''
        lr = self.learning_rate
        for fr, to, new_quality, new_normalizer in zip(
            path[:-1], path[1:], new_qualities, new_normalizers):
            self.normalizers[fr] = (1-lr)*self.normalizers[fr] + lr*new_normalizer
            self.qualities[fr, to] = (1-lr)*self.qualities[fr, to] + lr*new_quality
    
    async def train_for_one_rollout(self, start_city_id):
        '''
        对一次 rollout 的结果进行训练的流程
        '''
        path, action_probs, rewards = self.rollout(start_city_id=start_city_id)
        new_qualities, new_normalizers = self.calc_updates_for_one_rollout(path, action_probs, rewards)
        self.update(path, new_qualities, new_normalizers)

    async def train_for_one_epoch(self):
        '''
        对一个 epoch 的结果进行训练的流程，
        一个 epoch 对应于从每个 city 出发进行一次 rollout
        '''
        tasks = [self.train_for_one_rollout(start_city_id) for start_city_id in range(self.num_cities)]
        await asyncio.gather(*tasks)

    async def train(self, num_epochs=1000, display=True):
        '''
        总训练流程
        '''
        for epoch in range(num_epochs):
            await self.train_for_one_epoch()
            if display:
                print(epoch)
                

    # def draw(self, epoch):
    #     '''
    #     绘图
    #     '''
    #     _ = plt.scatter(*self.cities)
    #     for fr, to in zip(self.best_path[:-1], self.best_path[1:]):
    #         x1, y1 = self.cities[:, fr]
    #         x2, y2 = self.cities[:, to]
    #         dx, dy = x2-x1, y2-y1
    #         plt.arrow(x1, y1, dx, dy, width=0.01*min(self.map_size), 
    #                   edgecolor='orange', facecolor='white', animated=True, 
    #                   length_includes_head=True)
    #     nrs = np.exp(self.qualities)
    #     for i in range(self.num_cities):
    #         nrs[i, i] = 0
    #     gap = np.abs(np.exp(self.normalizers) - nrs.sum(-1)).mean()
    #     plt.title(f'epoch {epoch}: path length = {self.best_path_length:.2f}, normalizer error = {gap:.3f}')
    #     plt.savefig('tsp.png')
    #     plt.show()
    #     clear_output(wait=True)

In [109]:
import numpy as np

class TSP:
    def __init__(self, distance_matrix, city_indices, city_scores, city_prices, starting_city_idx=0, 
                 learning_rate=0.1, discount_factor=0.9, epsilon=1.0, 
                 epsilon_decay=0.995, min_epsilon=0.01, alpha=0.9, beta=0.2):
        
        self.full_distance_matrix = distance_matrix
        self.city_indices = city_indices
        self.city_scores = city_scores  # 城市的景点评分
        self.city_prices = city_prices  # 城市的景点价格
        self.num_cities = len(city_indices)
        self.starting_city_idx = city_indices[starting_city_idx]
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.alpha = alpha  # 用于调节距离和景点评分的权重
        self.beta = beta  # 用于调节价格的权重
        
        # 创建子距离矩阵
        self.distance_matrix = distance_matrix[np.ix_(city_indices, city_indices)]
        self.q_table = np.zeros((self.num_cities, self.num_cities))
        self.best_path = []
        self.best_path_length = np.inf
    
    def choose_next_city(self, current_city, visited):
        available_cities = [i for i in range(self.num_cities) if i not in visited]
        
        if len(available_cities) == 0:  # 如果没有可选城市，则选择起始城市
            return visited[0]  # 回到起始城市
        
        if np.random.rand() < self.epsilon:  # Exploration
            return np.random.choice(available_cities)
        else:  # Exploitation
            return max(available_cities, key=lambda x: self.q_table[current_city][x])
    
    def update_q_table(self, current_city, next_city, reward):
        # Q-learning 更新规则
        best_next_action = np.max(self.q_table[next_city])  # 获取下一个城市的最大Q值
        self.q_table[current_city][next_city] += self.learning_rate * (
            reward + self.discount_factor * best_next_action - self.q_table[current_city][next_city]
        )
    
    def calculate_reward(self, current_city, next_city):
        # 奖励函数设计：结合距离、景点评分和价格
        distance_reward = -self.alpha * self.distance_matrix[current_city][next_city]  # 负距离奖励
        score_reward = (1 - self.alpha) * self.city_scores[next_city]  # 景点评分奖励
        price_penalty = self.beta * self.city_prices[next_city]  # 价格惩罚，价格越高惩罚越大
        return distance_reward + score_reward - price_penalty
    
    def train(self, num_episodes=1000):
        for episode in range(num_episodes):
            visited = []
            current_city = 0  # Start from the first city in the submatrix
            total_distance = 0
            
            while len(visited) < self.num_cities:
                next_city = self.choose_next_city(current_city, visited)
                visited.append(next_city)
                total_distance += self.distance_matrix[current_city][next_city]
                reward = self.calculate_reward(current_city, next_city)  # 使用新的奖励函数
                self.update_q_table(current_city, next_city, reward)
                current_city = next_city
            
            # 完成一个循环（返回起始城市）
            total_distance += self.distance_matrix[current_city][0]
            
            # 更新最优路径，确保路径不重复并且是回路
            if total_distance < self.best_path_length:
                self.best_path_length = total_distance
                self.best_path = [self.city_indices[i] for i in visited]
                
                # 确保路径形成回路，不重复添加起始城市
                if self.best_path[0] != self.best_path[-1]:
                    self.best_path.append(self.best_path[0])  # 添加起始城市，确保回路
                
            # Decay epsilon to reduce exploration over time
            self.epsilon = max(self.epsilon * self.epsilon_decay, self.min_epsilon)
        
        return self.best_path, self.best_path_length


In [ ]:
df = pd.read_csv('shanxi5ASpot.csv')  
city_scores = df['评分'].tolist()    
city_prices = df['价格'].tolist()  

In [111]:
city_indices = [9, 1, 3,6]  # 选择城市 0, 1, 3  

tsp_solver = TSP(distance_matrix,city_indices,city_scores,city_prices)  

best_path, best_length = tsp_solver.train()  
best_path_cities = [all_cities[idx] for idx in best_path]

# 输出具体的城市路径
print("Best Path:", best_path_cities)
# print("Best Path:", best_path)  
print("Best Path Length:", best_length)

Best Path: ['山西王家大院', '小西天', '应县木塔', '云冈石窟', '山西王家大院']
Best Path Length: 1095.3000000000002
